
# Wearables iPython Notebook

Wearables Device EcoSystem Framework. (WDESF)

The scope of this development is to connect to users wearable devices, import the data, create a standard structure for the data, and data analysis.

Required:
Python 3.4 - Using VirtualEnv
mpld3 installed

The purpose of this document is to develop a workflow for the wearables scripts and programs.

Getting access to the data:
There are many options to connect to the Data:
1) Validic ($15,000 a year academic - limited to 1,000 participants)
2) Use OpenMHealth as data device connection (Limited on devices)
3) Direct data - Getting data from providers (Basis)



In [1]:
##Check Python Version
import sys
print(sys.version)

2.7.10 (default, Jul 14 2015, 19:46:27) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.39)]


## Get Data from BigQuery

In [2]:
import argparse

from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.client import GoogleCredentials


def main(project_id):
    # Grab the application's default credentials from the environment.
    credentials = GoogleCredentials.get_application_default()
    # Construct the service object for interacting with the BigQuery API.
    bigquery_service = build('bigquery', 'v2', credentials=credentials)

    try:
        query_request = bigquery_service.jobs()
        query_data = {
            'query': (
                'SELECT TOP(corpus, 10) as title, '
                'COUNT(*) as unique_words '
                'FROM [publicdata:samples.shakespeare];')
        }

        query_response = query_request.query(
            projectId=project_id,
            body=query_data).execute()

        print('Query Results:')
        for row in query_response['rows']:
            print('\t'.join(field['v'] for field in row['f']))

    except HttpError as err:
        print('Error: {}'.format(err.content))
        raise err


if __name__ == '__main__':
    main(project_id="gbsc-gcp-project-wearables")

IOError: [Errno 13] Permission denied

In [ ]:
import csv
import datetime
import numpy as np
import pandas as pd
import pdb
import pylab
import matplotlib.pyplot as plt

%matplotlib inline

basis_b1_hr = []
basis_peak_hr = []
time = []

basis_b1_hr_dict = dict()
basis_peak_hr_dict = dict()
time_dict = dict()

with open('/Users/dsalins/Documents/SVNRepositories/Stanford_Wearables/data/mps_basis_b1_peak_combined.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',') 
    for i, row in enumerate(spamreader):
        if i > 0:
            try:
                #time.append(datetime.datetime(row[0]))
                basis_b1_hr.append(float(row[3]))
                basis_peak_hr.append(float(row[8]))
                #time_dict[datetime.datetime(row[0])] = [row[3], row[8]]
            except:
                pass

import pdb ; pdb.set_trace()
# Basis B1 vs Basis Peak, fill the series
basis_b1 = pd.Series(basis_b1_hr)
basis_peak = pd.Series(basis_peak_hr)

pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 400)
pd.set_option('display.mpl_style', 'default')

font = {'size'   : 10}
matplotlib.rc('font', **font)

print("Plotting...")
basis_b1.plot()
basis_peak.plot()
plt.xlabel('Date/Time')
plt.ylabel('Heart rate')
plt.show()


# Schema Data Import

##Basis_B1_Import_Schema
time:timestamp,user_id:string,hr:string,accel_magnitude:string,gsr:string,skin_temp:string,activity_level:string,activity:string,steps:string,calories:string,device_model:string

##Basis_Peak_Import_Schema
date:timestamp,calories:float,gsr:float,heart_rate:string,skin_temp:float,steps:float

##GDEvice SChema
time:timestamp,pressure:string,altitude:string,battery_voltage:string,humidity:string,pulse_rate:string,activity:string,ambient_light:string,battery_status:string,flash_used:string,skin_temp:string,skin_response:string,subsystem_status:string,raw_cap_sense:string,ambient_temp:string,full_crash_log:string,firehose:string,composite_pulse_rate:string,pressure_2:float,altitude_2:string,battery_voltage_2:string,humidity_2:float,pulse_rate_2:float,activity_2:string,ambient_light_2:string,battery_status_2:string,flash_used_2:string,skin_temp_2:float,skin_response_2:float,subsystem_status_2:string,raw_cap_sense_2:string,ambient_temp_2:float,crash_log:string,full_crash_log_2:string

##MyFitnessPal - Schema
user_id:string,username:string,food_id:string,entry_date:string,amount:string,description:string,calories:float,total_fat:float,polyunsaturated_fat:float,monounsaturated_fat:float,trans_fat:float,cholesterol:float,sodium:float,potassium:float,carbs:float,fiber:float,sugar:float,protein:float,vitamin_a:float,vitamin_c:float,calcium:float,iron:float

##Cronometer
Day:TIMESTAMP,Food_Name:STRING,Amount:FLOAT,Energy:FLOAT,Alcohol:FLOAT,Ash:FLOAT,Caffeine:FLOAT,Water:FLOAT,B1_Thiamine:FLOAT,B12_Cobalamin:FLOAT,B2_Riboflavin:FLOAT,B3_Niacin:FLOAT,B5_Pantothenic_Acid:FLOAT,B6_Pyridoxine:FLOAT,Folate:FLOAT,Vitamin_A:FLOAT,Vitamin_C:FLOAT,Vitamin_D:FLOAT,Vitamin_E:FLOAT,Vitamin_K:FLOAT,Calcium:FLOAT,Copper:FLOAT,Iron:FLOAT,Magnesium:FLOAT,Manganese:FLOAT,Phosphorus:FLOAT,Potassium:FLOAT,Selenium:FLOAT,Sodium:FLOAT,Zinc:FLOAT,Carbs:FLOAT,Fiber:FLOAT,Starch:FLOAT,Sugars:FLOAT,Fat:FLOAT,Cholesterol:FLOAT,Monounsaturated:FLOAT,Omega3:FLOAT,Omega6:FLOAT,Polyunsaturated:FLOAT,Saturated:FLOAT,TransFats:FLOAT,Cystine:FLOAT,Histidine:FLOAT,Isoleucine:FLOAT,Leucine:FLOAT,Lysine:FLOAT,Methionine:FLOAT,Phenylalanine:FLOAT,Protein:FLOAT,Threonine:FLOAT,Tryptophan:FLOAT,Tyrosine:FLOAT,Valine:FLOAT

##Moves - Schema
type:string
start_day_time:timestamp
end_day_time:timestamp
activity:string
group:string
duration:float
distance:float
steps:float
manual:bool
calories:float

In [3]:
#!/usr/bin/env python
%matplotlib inline
"""
Compute the cross spectral density of two signals
"""
import numpy as np
import matplotlib.pyplot as plt

# make a little extra space between the subplots
plt.subplots_adjust(wspace=0.5)

dt = 0.01
t = np.arange(0, 30, dt)
nse1 = np.random.randn(len(t))                 # white noise 1
nse2 = np.random.randn(len(t))                 # white noise 2
r = np.exp(-t/0.05)

cnse1 = np.convolve(nse1, r, mode='same')*dt   # colored noise 1
cnse2 = np.convolve(nse2, r, mode='same')*dt   # colored noise 2

# two signals with a coherent part and a random part
s1 = 0.01*np.sin(2*np.pi*10*t) + cnse1
s2 = 0.01*np.sin(2*np.pi*10*t) + cnse2

plt.subplot(211)
plt.plot(t, s1, 'b-', t, s2, 'g-')
plt.xlim(0,5)
plt.xlabel('time')
plt.ylabel('s1 and s2')
plt.grid(True)

plt.subplot(212)
cxy, f = plt.csd(s1, s2, 256, 1./dt)
plt.ylabel('CSD (db)')
plt.show()

In [1]:
from pylab import plotfile, show, gca
import matplotlib.cbook as cbook

#fname = cbook.get_sample_data('/Users/dsalins/Documents/SVNRepositories/Stanford_Wearables/data/mps_basis_b1_peak_combined.csv', asfileobj=False)
fname2 = cbook.get_sample_data('msft.csv', asfileobj=False)

#test 5; single subplot
#plotfile(fname, ('wearable_mps_basis_b1_date', 'wearable_mps_basis_peak_heart_rate', 'wearable_mps_basis_b1_hr'), subplots=False)
plotfile(fname2, ('date', 'open', 'high', 'low', 'close'), subplots=False)

show()

In [2]:
import datetime
import numpy as np
import matplotlib.colors as colors
import matplotlib.finance as finance
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

%matplotlib inline

startdate = datetime.date(2006,1,1)
today = enddate = datetime.date.today()
ticker = 'GOOG'


fh = finance.fetch_historical_yahoo(ticker, startdate, enddate)
# a numpy record array with fields: date, open, high, low, close, volume, adj_close)

r = mlab.csv2rec(fh); fh.close()
r.sort()


def moving_average(x, n, type='simple'):
    """
    compute an n period moving average.

    type is 'simple' | 'exponential'

    """
    x = np.asarray(x)
    if type=='simple':
        weights = np.ones(n)
    else:
        weights = np.exp(np.linspace(-1., 0., n))

    weights /= weights.sum()


    a =  np.convolve(x, weights, mode='full')[:len(x)]
    a[:n] = a[n]
    return a

def relative_strength(prices, n=14):
    """
    compute the n period relative strength indicator
    http://stockcharts.com/school/doku.php?id=chart_school:glossary_r#relativestrengthindex
    http://www.investopedia.com/terms/r/rsi.asp
    """

    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi

def moving_average_convergence(x, nslow=26, nfast=12):
    """
    compute the MACD (Moving Average Convergence/Divergence) using a fast and slow exponential moving avg'
    return value is emaslow, emafast, macd which are len(x) arrays
    """
    emaslow = moving_average(x, nslow, type='exponential')
    emafast = moving_average(x, nfast, type='exponential')
    return emaslow, emafast, emafast - emaslow

plt.rc('axes', grid=True)
plt.rc('grid', color='0.75', linestyle='-', linewidth=0.5)

textsize = 9
left, width = 0.1, 0.8
rect1 = [left, 0.7, width, 0.2]
rect2 = [left, 0.3, width, 0.4]
rect3 = [left, 0.1, width, 0.2]


fig = plt.figure(facecolor='white')
axescolor  = '#f6f6f6'  # the axes background color

ax1 = fig.add_axes(rect1, axisbg=axescolor)  #left, bottom, width, height
ax2 = fig.add_axes(rect2, axisbg=axescolor, sharex=ax1)
ax2t = ax2.twinx()
ax3  = fig.add_axes(rect3, axisbg=axescolor, sharex=ax1)

### plot the relative strength indicator
prices = r.adj_close
rsi = relative_strength(prices)
fillcolor = 'darkgoldenrod'

ax1.plot(r.date, rsi, color=fillcolor)
ax1.axhline(70, color=fillcolor)
ax1.axhline(30, color=fillcolor)
ax1.fill_between(r.date, rsi, 70, where=(rsi>=70), facecolor=fillcolor, edgecolor=fillcolor)
ax1.fill_between(r.date, rsi, 30, where=(rsi<=30), facecolor=fillcolor, edgecolor=fillcolor)
ax1.text(0.6, 0.9, '>70 = overbought', va='top', transform=ax1.transAxes, fontsize=textsize)
ax1.text(0.6, 0.1, '<30 = oversold', transform=ax1.transAxes, fontsize=textsize)
ax1.set_ylim(0, 100)
ax1.set_yticks([30,70])
ax1.text(0.025, 0.95, 'RSI (14)', va='top', transform=ax1.transAxes, fontsize=textsize)
ax1.set_title('%s daily'%ticker)

### plot the price and volume data
dx = r.adj_close - r.close
low = r.low + dx
high = r.high + dx

deltas = np.zeros_like(prices)
deltas[1:] = np.diff(prices)
up = deltas>0
ax2.vlines(r.date[up], low[up], high[up], color='black', label='_nolegend_')
ax2.vlines(r.date[~up], low[~up], high[~up], color='black', label='_nolegend_')
ma20 = moving_average(prices, 20, type='simple')
ma200 = moving_average(prices, 200, type='simple')

linema20, = ax2.plot(r.date, ma20, color='blue', lw=2, label='MA (20)')
linema200, = ax2.plot(r.date, ma200, color='red', lw=2, label='MA (200)')


last = r[-1]
s = '%s O:%1.2f H:%1.2f L:%1.2f C:%1.2f, V:%1.1fM Chg:%+1.2f' % (
    today.strftime('%d-%b-%Y'),
    last.open, last.high,
    last.low, last.close,
    last.volume*1e-6,
    last.close-last.open )
t4 = ax2.text(0.3, 0.9, s, transform=ax2.transAxes, fontsize=textsize)

props = font_manager.FontProperties(size=10)
leg = ax2.legend(loc='center left', shadow=True, fancybox=True, prop=props)
leg.get_frame().set_alpha(0.5)


volume = (r.close*r.volume)/1e6  # dollar volume in millions
vmax = volume.max()
poly = ax2t.fill_between(r.date, volume, 0, label='Volume', facecolor=fillcolor, edgecolor=fillcolor)
ax2t.set_ylim(0, 5*vmax)
ax2t.set_yticks([])


### compute the MACD indicator
fillcolor = 'darkslategrey'
nslow = 26
nfast = 12
nema = 9
emaslow, emafast, macd = moving_average_convergence(prices, nslow=nslow, nfast=nfast)
ema9 = moving_average(macd, nema, type='exponential')
ax3.plot(r.date, macd, color='black', lw=2)
ax3.plot(r.date, ema9, color='blue', lw=1)
ax3.fill_between(r.date, macd-ema9, 0, alpha=0.5, facecolor=fillcolor, edgecolor=fillcolor)


ax3.text(0.025, 0.95, 'MACD (%d, %d, %d)'%(nfast, nslow, nema), va='top',
         transform=ax3.transAxes, fontsize=textsize)

#ax3.set_yticks([])
# turn off upper axis tick labels, rotate the lower ones, etc
for ax in ax1, ax2, ax2t, ax3:
    if ax!=ax3:
        for label in ax.get_xticklabels():
            label.set_visible(False)
    else:
        for label in ax.get_xticklabels():
            label.set_rotation(30)
            label.set_horizontalalignment('right')

    ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')



class MyLocator(mticker.MaxNLocator):
    def __init__(self, *args, **kwargs):
        mticker.MaxNLocator.__init__(self, *args, **kwargs)

    def __call__(self, *args, **kwargs):
        return mticker.MaxNLocator.__call__(self, *args, **kwargs)

# at most 5 ticks, pruning the upper and lower so they don't overlap
# with other ticks
#ax2.yaxis.set_major_locator(mticker.MaxNLocator(5, prune='both'))
#ax3.yaxis.set_major_locator(mticker.MaxNLocator(5, prune='both'))

ax2.yaxis.set_major_locator(MyLocator(5, prune='both'))
ax3.yaxis.set_major_locator(MyLocator(5, prune='both'))

plt.show()